In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Depression"
cohort = "GSE99725"

# Input paths
in_trait_dir = "../../input/GEO/Depression"
in_cohort_dir = "../../input/GEO/Depression/GSE99725"

# Output paths
out_data_file = "../../output/preprocess/Depression/GSE99725.csv"
out_gene_data_file = "../../output/preprocess/Depression/gene_data/GSE99725.csv"
out_clinical_data_file = "../../output/preprocess/Depression/clinical_data/GSE99725.csv"
json_path = "../../output/preprocess/Depression/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptomic Signaling Pathways involved in a naturalistic model  of Inflammation-related Major Depressive Disorder and its remission"
!Series_summary	"This study aimed at identifying molecular biomarkers specific to inflammation-related subtype of MDD in order to improve diagnosis and treatment. For this, we performed whole-genome expression profiling from peripheral blood in a naturalistic model of inflammation-associated MDD represented by comorbid depression in obese patients. "
!Series_overall_design	"Depressed patients were diagnosed with the Mini-International Neuropsychiatric Interview and the 10-item, clinician administered, Montgomery-Asberg Depression Rating Scale. From a cohort of 100 massively obese patients we selected 33 of them for transcriptomic analysis with 24 patients that were again analyzed 4-12 months after bariatric surgery. "
Sample Characteristics Dictionary:
{0: ['patient: CB291013', 'patient: TP100414', 'patient: JDF2

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# The dataset appears to contain transcriptomic data based on background information
# "whole-genome expression profiling from peripheral blood"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Trait (Depression): From the sample characteristic dictionary, key 2 with 'MADRS: A', 'MADRS: B'
# MADRS is Montgomery-Asberg Depression Rating Scale which is used to measure depression severity
trait_row = 2

# Age: Not explicitly available in the sample characteristics dictionary
age_row = None

# Gender: Not explicitly available in the sample characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert MADRS ratings to binary depression status.
    MADRS: A - likely represents patients with high MADRS scores (depressed)
    MADRS: B - likely represents patients with low MADRS scores (not depressed or remission)
    """
    if isinstance(value, str):
        value = value.strip()
        if 'MADRS: A' in value:
            return 1  # Depressed
        elif 'MADRS: B' in value:
            return 0  # Not depressed or in remission
    return None

def convert_age(value):
    """
    Convert age values to continuous data type.
    Not used in this dataset as age information is not available.
    """
    return None

def convert_gender(value):
    """
    Convert gender values to binary (0 for female, 1 for male).
    Not used in this dataset as gender information is not available.
    """
    return None

# 3. Save Metadata
# Trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Proceed only if trait_row is not None
if trait_row is not None:
    # Assuming clinical_data was loaded in a previous step
    try:
        # Get the clinical data path
        clinical_data_file = os.path.join(in_cohort_dir, "clinical_data.csv")
        clinical_data = pd.read_csv(clinical_data_file)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview and save the clinical data
        print("Preview of selected clinical features:")
        print(preview_df(selected_clinical_df))
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save to CSV
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except FileNotFoundError:
        print("Clinical data file not found. Skip clinical feature extraction.")


Clinical data file not found. Skip clinical feature extraction.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Depression/GSE99725/GSE99725_series_matrix.txt.gz
Gene data shape: (27202, 57)
First 20 gene/probe identifiers:
Index(['A_19_P00315452', 'A_19_P00315459', 'A_19_P00315493', 'A_19_P00315506',
       'A_19_P00315524', 'A_19_P00315528', 'A_19_P00315529', 'A_19_P00315550',
       'A_19_P00315551', 'A_19_P00315581', 'A_19_P00315583', 'A_19_P00315584',
       'A_19_P00315593', 'A_19_P00315601', 'A_19_P00315603', 'A_19_P00315649',
       'A_19_P00315651', 'A_19_P00315668', 'A_19_P00315691', 'A_19_P00315693'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers
# These identifiers (starting with A_19_P) are Agilent probe IDs, not human gene symbols.
# They need to be mapped to standard gene symbols for analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# Check if there are any platforms defined in the SOFT file that might contain annotation data
with gzip.open(soft_file, 'rt') as f:
    soft_content = f.read()

# Look for platform sections in the SOFT file
platform_sections = re.findall(r'^!Platform_title\s*=\s*(.+)$', soft_content, re.MULTILINE)
if platform_sections:
    print(f"Platform title found: {platform_sections[0]}")

# Try to extract more annotation data by reading directly from the SOFT file
# Look for lines that might contain gene symbol mappings
symbol_pattern = re.compile(r'ID_REF\s+Symbol|ID\s+Gene Symbol', re.IGNORECASE)
annotation_lines = []
with gzip.open(soft_file, 'rt') as f:
    for line in f:
        if symbol_pattern.search(line):
            annotation_lines.append(line)
            # Collect the next few lines to see the annotation structure
            for _ in range(10):
                annotation_lines.append(next(f, ''))

if annotation_lines:
    print("Found potential gene symbol mappings:")
    for line in annotation_lines:
        print(line.strip())

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("\nGene annotation preview:")
print(preview_df(gene_annotation, n=10))

# If we need an alternative source of mapping, check if there are any other annotation files in the cohort directory
cohort_files = os.listdir(in_cohort_dir)
annotation_files = [f for f in cohort_files if 'annotation' in f.lower() or 'platform' in f.lower()]
if annotation_files:
    print("\nAdditional annotation files found in the cohort directory:")
    for file in annotation_files:
        print(file)


Platform title found: Agilent-039494 SurePrint G3 Human GE v2 8x60K Microarray 039381 (Probe Name version)



Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220', 'A_33_P3236322', 'A_33_P3319925', 'A_21_P0000509', 'A_21_P0000744', 'A_24_P215804'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220', 'A_33_P3236322', 'A_33_P3319925', 'A_21_P0000509', 'A_21_P0000744', 'A_24_P215804'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466', nan, 'XM_001133269', 'NR_024244', 'NR_038269', 'NM_016951'], 'GB_ACC': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466', 'AK128005', 'XM_001133269', 'NR_024244', 'NR_038269', 'NM_016951'], 'LOCUSLINK_ID': [nan, nan, 50865.0, 23704.0, 128861.0, 100129869.0, 730249.0, nan, nan, 51192.0], 'GENE_SYMBOL': [nan, nan, 'HEBP1', 'KCNE4', 'BPIFA3', 'LOC100129869', 'IRG1', 'SNAR-G2', 'LOC100506844', 'CKLF'], 'GENE_NAME': [nan, nan, 'heme binding protein 1', 'potassium vol

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the keys for gene identifiers and gene symbols
# The 'ID' column in the gene_annotation matches the probe identifiers in gene_data
# The 'GENE_SYMBOL' column contains the corresponding gene symbols
prob_col = 'ID'
gene_col = 'GENE_SYMBOL'

# 2. Get a gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("First few rows of mapping dataframe:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few rows of gene-mapped expression data:")
print(gene_data.head())

# Create the output directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the gene expression data to a CSV file
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Mapping dataframe shape: (46204, 2)
First few rows of mapping dataframe:
              ID          Gene
2   A_23_P117082         HEBP1
3  A_33_P3246448         KCNE4
4  A_33_P3318220        BPIFA3
5  A_33_P3236322  LOC100129869
6  A_33_P3319925          IRG1
Gene expression data shape after mapping: (14534, 57)
First few rows of gene-mapped expression data:
          GSM2650879  GSM2650880  GSM2650881  GSM2650882  GSM2650883  \
Gene                                                                   
A1BG       22.991629   22.721831   23.722127   23.393494   23.642667   
A1BG-AS1   30.547082   29.264743   30.395788   29.542070   29.606270   
A2LD1      54.674383   55.100095   56.005337   56.411266   55.148358   
A4GALT     26.780564   27.006581   27.308758   28.217594   27.636441   
AAAS       57.085719   56.312345   56.760835   58.313588   57.552408   

          GSM2650884  GSM2650885  GSM2650886  GSM2650887  GSM2650888  ...  \
Gene                                                      

Gene expression data saved to ../../output/preprocess/Depression/gene_data/GSE99725.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data - we already did this in step 6
# No need to do it again
print(f"Gene data already normalized and saved to {out_gene_data_file}")

# 2. Need to recreate clinical data since it wasn't properly saved in step 2
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Extract clinical features
def convert_trait(value):
    """
    Convert MADRS ratings to binary depression status.
    MADRS: A - likely represents patients with high MADRS scores (depressed)
    MADRS: B - likely represents patients with low MADRS scores (not depressed or remission)
    """
    if isinstance(value, str):
        value = value.strip()
        if 'MADRS: A' in value:
            return 1  # Depressed
        elif 'MADRS: B' in value:
            return 0  # Not depressed or in remission
    return None

# Use previously defined trait_row = 2 from step 2
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=2,
    convert_trait=convert_trait,
    age_row=None,
    convert_age=None,
    gender_row=None, 
    convert_gender=None
)

print(f"Selected clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save clinical data for future reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 2. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from peripheral blood of obese patients with and without depression."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Gene data already normalized and saved to ../../output/preprocess/Depression/gene_data/GSE99725.csv
Selected clinical data shape: (1, 57)
Clinical data preview:
{'GSM2650879': [1.0], 'GSM2650880': [1.0], 'GSM2650881': [0.0], 'GSM2650882': [0.0], 'GSM2650883': [1.0], 'GSM2650884': [0.0], 'GSM2650885': [1.0], 'GSM2650886': [1.0], 'GSM2650887': [1.0], 'GSM2650888': [1.0], 'GSM2650889': [1.0], 'GSM2650890': [0.0], 'GSM2650891': [0.0], 'GSM2650892': [0.0], 'GSM2650893': [0.0], 'GSM2650894': [0.0], 'GSM2650895': [1.0], 'GSM2650896': [1.0], 'GSM2650897': [1.0], 'GSM2650898': [1.0], 'GSM2650899': [0.0], 'GSM2650900': [0.0], 'GSM2650901': [0.0], 'GSM2650902': [1.0], 'GSM2650903': [1.0], 'GSM2650904': [1.0], 'GSM2650905': [1.0], 'GSM2650906': [1.0], 'GSM2650907': [0.0], 'GSM2650908': [0.0], 'GSM2650909': [0.0], 'GSM2650910': [0.0], 'GSM2650911': [0.0], 'GSM2650912': [0.0], 'GSM2650913': [0.0], 'GSM2650914': [0.0], 'GSM2650915': [1.0], 'GSM2650916': [0.0], 'GSM2650917': [1.0], 'GSM2650918': [1.0]

Data shape after handling missing values: (57, 14535)
For the feature 'Depression', the least common label is '0.0' with 26 occurrences. This represents 45.61% of the dataset.
The distribution of the feature 'Depression' in this dataset is fine.



Linked data saved to ../../output/preprocess/Depression/GSE99725.csv
